# 04. Transter Learning with TensorFlow Part 1: Feature Extraction

이 장에서 중요하게 다루는 개념들.

1. 전이 학습
- 정의
    - 다른 모델의 패턴을 가져와서 이용하는 것
- 장점.
    - 현존하는 신경망 구조로 비슷한 문제를 해결할 수 있음

2. callbacks
- **TensorBoard를 활용한 실험 추적(Experiment tracking)**
    - 여러 모델의 성능을 기록하고 이를 TensorBoard(신경망 파라미터를 시각적으로 확인할 수 있는 대시보드)에서 비교 및 시각함
    - 다양한 모델이 같은 데이터에서 어떻게 작동하는지 비교할 때 유용

- **모델 체크포인트 저장(Model checkpointing)**
    - 훈련 중간중간 모델을 저장할 수 있어서, 훈련을 중단했다가 다시 이어서 진행할 수 있음
    - 훈련 시간이 오래 걸리는 경우 유용

- **얼리 스토핑(Early stopping)**
    - 일정 시간 동안 모델을 훈련시킨 뒤, 성능 향상이 없으면 자동으로 훈련을 멈춥니다.
    - 데이터셋이 크고 훈련 시간이 얼마나 걸릴지 모를 때 유용합니다.

## What we're going to cover

1. Introduce transfer learning (a way to beat all of our old self-built models)
2. Using a smaller dataset to experiment faster (10% of training samples of 10 classes of food)
3. Build a transfer learning feature extraction model using TensorFlow Hub
4. Introduce the TensorBoard callback to track model training results
5. Compare model results using TensorBoard

In [24]:
# Add tiemstamp
import datetime
print(f"Notebook last run (end-to-end) : {datetime.datetime.now()}")

Notebook last run (end-to-end) : 2025-05-29 14:50:11.259866


In [25]:
# Are we using a GPU?
!nvidia_smi

/bin/bash: line 1: nvidia_smi: command not found


## Downloading and becoming one with the data

In [26]:
# Get data (10% of labels)
import zipfile

#Download data
!wget  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unaip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2025-05-29 14:50:11--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   177MB/s    in 0.9s    

2025-05-29 14:50:12 (177 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



KeyboardInterrupt: 

In [ ]:
# How many images in each folder?
import os

# Walk though 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}.")

In [23]:
## Create data loaders (preparing the data)
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")


test_data_10_percent = train_datagen.flow_from_directory(test_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")



Training images:
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


# Setting up callbacks (things to run whilst our model trains)

In [ ]:
# Create tensorboard callback (functionized because need to create a new one for each model)

import datetime
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dri = log_dir
    )
    print(f"Saving TensorBoard log file to : {log_dir}")
    return tensorboard_callback

## Creating model using TensofFlow Hub

TesorFlow Hub에서 사용할 2 모델:
1. ResNetV2 - a state of the art computer vision model architecture from 2016
2. EfficientNet - a state of the art computer vision architecture from 2019

In [22]:
# Install TensorFlow and TensorFlow Hub to ensure compatibility
!pip install --upgrade tensorflow tensorflow-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 954.5 kB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.18.1 which is incompatible.


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Resnet 50 V2 feature vector
resnet_url  = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EffiocientNetB) feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [ ]:
def create_model(model_url, num_classes = 10):
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             trainable=False, # freeze the underlying patterns
                                             name='feature_extraction_layer',
                                             input_shape = IMAGE_SHAPE+(3,)) # define the input image shape

    # Create our own model
    model = tf.keras.Sequential([
        feature_extractor_layer, # use the feature extraction layer as the base
        layers.Dense(num_classes, activation='softmax', name ='output_layer') # create our own output layer
        ])

    return model

In [ ]:
# Create model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

# Compile
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
# Fit the model
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs = 5,
                                  steps_per_epoch = len(train_data_10_percent),
                                  validation_date = test_data,
                                  validation_steps=len(test_data),
                                  #Add tensofboard callback to model( callbacks parameter takes a list)
                                  callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub", # save experiment logs here
                                                                           experiment_name = 'resnet50V2')] # name of log files
                                  )

In [1]:
import matplotlib.pyplot as plt

# Plot the validation and tranining data separately
def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label="training_accuracy")
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();


In [ ]:
plot_loss_curves(resnet_history)

In [ ]:
# Resnet summary
resnet_model.summary()

In [2]:
# Create model
efficientnet_model = create_model(model_url=efficientnet_url, # use EfficientNetB0 TensorFlow Hub URL
                                  num_classes = train_data_10_percent.num_classes)

# Complie EfficientNet model
efficientnet_model.compile(loss='categorical_crossentropy',
                           optimizer = tf.kears.optimizers.Adam(),
                           metrics=['accuracy'])

# Fit EfficientNet model
efficietnet_history = efficietnnet_model.fit(train_data_10_percent, # only use 10 % of training data
                                             epochs=5,
                                             steps_per_epoch = len(train_data_10_percent),
                                             validation_data = test_data,
                                             validation_steps = len(test_data),
                                             callbacks = [create_tensorboard_callback(dir_name='tensorflow_hub',
                                                                                      #Track logs under different experiment name
                                                                                      experiment_name = 'efficientnetB0')])

NameError: name 'create_model' is not defined

In [ ]:
plot_loss_curves(efficientnet_history)

In [ ]:
efficientnet_model.summary()

## Uploading experiments to TensorBoard

‼️‼️‼️‼️‼️‼️‼️‼️‼️
- --logdir is the target upload directory
- --name is the name of the experiment
- --description is a brief description of the experiment
- --one_shot exits the TensorBoard uploader once uploading is finished

In [ ]:
# Upload TensorBoard dev recods
!tensorboard dev upload --logdir ./tensorflow_hub/\
    --name "EfficientNetB0 vs. ResNet50V2" \
    --description "Comparing two different TF Hub feature extraction models architectrues ustin 10% of training images" \
    --one_shot

## Listing esperiments you've saved to TensorBoard

In [ ]:
# Check out experiments
# !tensorboard dev list # uncomment to see

## Deleting experiments form TensorBoard

In [ ]:
# Delete an experiment
!tensorboard dev delete --experiment_id

In [ ]:
# Check to see if experiments still exist
# !tensorboard dev list # uncomment to see